Emailid - rachitrawat18.18je0663@pe.iitism.ac.in


---


Extractive method: There is no parameter learning in this method just the most important sentences in the document are picked up on the basis of scores, which combines to form the summary without losing context of the language.
 
**Function details:**
 
Text_summarizer function has been built to use it as an API. The function contains several secondary functions used for text-preprocessing,
calculating TF-IDF scores and calculating sentence score.
 
**parameters:**
 
Text = text string(can be of any length)
 
percentage = int(0-100), default is 50, percentage parameter decides length of the summary returned.
 
**modules required:**required modules can be imported/downloaded by running the cell below.
 
**Corpus used:**Nltk packaged is utilised here for corpus and text-preprocessing as stemming and lemmatization are necessary before creating scores.
 
**Method used:**TF-IDF
 
This particular method uses TF-IDF scores that are alloted to all the words. To give a sentence score, scores of words present in a specific sentence are added. Finally, sentences are arranged in order of scores and top few are picked up to generate summary.

In [ ]:
# Importing the Required Libraries
import nltk
nltk.download('stopwords')
import os
import re
import math
import numpy as np
import pandas as pd
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()
from nltk import tokenize
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# Reading text file and calling text_summarizer to get summary
df = pd.read_excel('TASK1.xlsx')
text = df['product descriptons'][20]

# Calling text_summarizer to get summary and topic of the product description
topic, summary = Text_summarizer(text, percentage = 40)
print(topic+':')
print("\n")
print(summary)

Androanagen:


It works by increasing blood flow to the hair follicles which further prevents hair loss and stimulates re-growth resulting in longer, thicker and increased numbers of hair.Androanagen Solution should only be applied directly to the scalp area in the amount, and in the way, specified on the label or by your doctor. This medication must be used continuously to maintain hair growth. This should not happen if you use it correctly and avoid contact with broken skin. If you get it in your eyes, mouth or broken skin, rinse thoroughly with plenty of water. Consult your doctor if you are breastfeeding mother.


In [ ]:
def Text_summarizer(text, percentage = 50):
  # Lemmatization
  def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

  # Stemming to get only root words 
  def stem_words(words):
      stemmed_words = []
      for word in words:
        stemmed_words.append(stemmer.stem(word))
      return stemmed_words

  # Removing special characters
  def remove_special_characters(text):
      regex = r'[^a-zA-Z0-9\s]'
      text = re.sub(regex,'',text)
      return text

  # Calculating Word frequency    
  def freq(words):
      words = [word.lower() for word in words]
      dict_freq = {}
      words_unique = []
      for word in words:
        if word not in words_unique:
            words_unique.append(word)
      for word in words_unique:
        dict_freq[word] = words.count(word)
      return dict_freq

  # part-of-speech tagging    
  def pos_tagging(text):
      pos_tag = nltk.pos_tag(text.split())
      pos_tagged_noun_verb = []
      for word,tag in pos_tag:
          if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
              pos_tagged_noun_verb.append(word)
      return pos_tagged_noun_verb

  # Calculating term-frequency    
  def tf_score(word,sentence):
      freq_sum = 0
      word_frequency_in_sentence = 0
      len_sentence = len(sentence)
      for word_in_sentence in sentence.split():
          if word == word_in_sentence:
              word_frequency_in_sentence = word_frequency_in_sentence + 1
      tf =  word_frequency_in_sentence/ len_sentence
      return tf

  # Calculating IDF    
  def idf_score(no_of_sentences,word,sentences):
      no_of_sentence_containing_word = 0
      for sentence in sentences:
          sentence = remove_special_characters(str(sentence))
          sentence = re.sub(r'\d+', '', sentence)
          sentence = sentence.split()
          sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
          sentence = [word.lower() for word in sentence]
          sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
          if word in sentence:
              no_of_sentence_containing_word = no_of_sentence_containing_word + 1
      idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
      return idf

  # Calculating tf-idf    
  def tf_idf_score(tf,idf):
      return tf*idf

  # getting tf-idf of a word    
  def word_tfidf(dict_freq,word,sentences,sentence):
      word_tfidf = []
      tf = tf_score(word,sentence)
      idf = idf_score(len(sentences),word,sentences)
      tf_idf = tf_idf_score(tf,idf)
      return tf_idf

  # getting sentence score    
  def sentence_importance(sentence,dict_freq,sentences):
      sentence_score = 0
      sentence = remove_special_characters(str(sentence)) 
      sentence = re.sub(r'\d+', '', sentence)
      pos_tagged_sentence = [] 
      no_of_sentences = len(sentences)
      pos_tagged_sentence = pos_tagging(sentence)
      for word in pos_tagged_sentence:
            if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                  word = word.lower()
                  word = wordlemmatizer.lemmatize(word)
                  sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
      return sentence_score
  
  # Getting topic
  splitted = text.split()
  topic = splitted[0]

  # Text preprocessing
  tokenized_sentence = tokenize.sent_tokenize(text)
  text = remove_special_characters(str(text))
  text = re.sub(r'\d+', '', text)
  tokenized_words_with_stopwords = word_tokenize(text)
  tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
  tokenized_words = [word for word in tokenized_words if len(word) > 1]
  tokenized_words = [word.lower() for word in tokenized_words]
  tokenized_words = lemmatize_words(tokenized_words)

  # word frequency
  word_freq = freq(tokenized_words)

  input_user = percentage
  no_of_sentences = int((input_user * len(tokenized_sentence))/100)

  c = 1
  sentence_with_importance = {}
  for sent in tokenized_sentence:
      sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
      sentence_with_importance[c] = sentenceimp
      c = c+1

  sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

  cnt = 0
  summary = []
  sentence_no = []
  for word_prob in sentence_with_importance:
      if cnt < no_of_sentences:
          sentence_no.append(word_prob[0])
          cnt = cnt+1
      else:
        break
  sentence_no.sort()
  cnt = 1
  for sentence in tokenized_sentence:
      if cnt in sentence_no:
        summary.append(sentence)
      cnt = cnt+1
  summary = " ".join(summary)
  
  return topic,summary